# Tensorflow 2 example -- LSTM

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
x1 = pd.Series(range(0,2400))
x2 = pd.Series(range(10,10+2400))
x3 = pd.Series(range(20,20+2400))
y  = pd.Series(range(100,100+2400))

In [3]:
data = pd.DataFrame({'x1':x1,'x2':x2,'x3':x3,'y':y})

In [4]:
batch_size = 100   
n_steps = 300         
n_inputs = 3      
n_outputs = 1     
trainEpoch = 50 
learning_rate = 0.001

In [5]:
class mydata():
    def __init__(self,data,batch_size,n_steps,n_inputs,n_outputs,y):
        self.batch_size = batch_size
        self.n_steps    = n_steps
        self.n_inputs    = n_inputs
        self.n_outputs  = n_outputs
        self.train_x = data.iloc[0:len(data)-n_steps,0:n_inputs].values.astype(np.float32)
        # 0:len(data)-n_step 保留 n_step 數量的資料   #0:n_inputs 使用前幾個變數
        self.train_y = data.iloc[0:len(data)-n_steps][y].values.astype(np.float32)
        self.test_x = data.iloc[len(data)-n_steps:len(data),0:n_inputs].values.astype(np.float32)
        #取出最後 n_step 筆
        self.test_y = data.iloc[len(data)-n_steps:len(data)][y].values.astype(np.float32)
        #return self.train_x, self.train_y, self.test_x, self.test_y
    
    def ts_Batch(self):
        X_train = []
        y_train = [] 
        for i in range(self.n_steps,len(self.train_x)+1):
            X_train.append(np.array(self.train_x[i-self.n_steps:i]).reshape([1,self.n_steps,self.n_inputs]) )
            y_train.append(np.array(self.train_y[i-self.n_steps:i]).reshape([1,self.n_steps,self.n_outputs]))
        
        
        if self.batch_size ==1:
            totalBatches = len(X_train)
            return X_train, y_train, totalBatches
        
        elif self.batch_size >1 :
            LSTM_x = []
            LSTM_y = []
            for i in range(0,len(X_train),self.batch_size):
                LSTM_x.append(np.array(X_train[i:i+self.batch_size]).reshape([-1,self.n_steps,self.n_inputs]).astype(np.float32))
                LSTM_y.append(np.array(y_train[i:i+self.batch_size]).reshape([-1,self.n_steps,self.n_outputs]).astype(np.float32))
            totalBatches = len(LSTM_x)

        return LSTM_x, LSTM_y, totalBatches

# Method 1


In [6]:
class LSTM_MODEL(tf.keras.Model):
    def __init__(self, batch_size, n_steps, n_inputs, n_outputs):
        super().__init__()
        tf.compat.v1.set_random_seed(123)
        self.batch_size = batch_size
        self.n_steps  = n_steps
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.cell = tf.keras.layers.RNN([tf.keras.layers.LSTMCell(100),tf.keras.layers.LSTMCell(50),tf.keras.layers.LSTMCell(25)], 
                                        return_sequences=True, return_state=True)
        self.dense1 = tf.keras.layers.Dense(units = 10)
        self.dense2 = tf.keras.layers.Dense(units = 5)
        self.dense3 = tf.keras.layers.Dense(units = n_outputs)
        
    def call(self,inputs):
        output ,sequence, state,st= self.cell(inputs)
        #print(output)
        output = self.dense1(output)
        output = self.dense2(output)
        output = self.dense3(output)
        #print(output)


        return output#,sequence, state

In [7]:
class LSTM_MODEL(tf.keras.Model):
    def __init__(self, batch_size, n_steps, n_inputs, n_outputs):
        super().__init__()
        tf.compat.v1.set_random_seed(123)
        self.batch_size = batch_size
        self.n_steps  = n_steps
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.cell = tf.keras.layers.RNN([tf.keras.layers.LSTMCell(100),tf.keras.layers.LSTMCell(40)], 
                                        return_sequences=True, return_state=True)
        self.dense1 = tf.keras.layers.Dense(units = 10)
        self.dense2 = tf.keras.layers.Dense(units = 5)
        self.dense3 = tf.keras.layers.Dense(units = n_outputs)
        
    def call(self,inputs):
        output ,sequence, state= self.cell(inputs)
        #print(output)
        output = self.dense1(output)
        output = self.dense2(output)
        output = self.dense3(output)
        #print(output)


        return output#,sequence, state

In [8]:
dataloader = mydata(data,batch_size,n_steps,n_inputs,n_outputs,'y')
model = LSTM_MODEL(batch_size, n_steps, n_inputs, n_outputs)
#dataloader = MnistLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)


In [9]:
%%time
X, y, num_batch = dataloader.ts_Batch()
with tf.device('/GPU:0'):
    for j in range(trainEpoch):
        for i in range(num_batch):
            with tf.GradientTape() as tape:
                y_pred = model(X[i])
                loss   = tf.keras.losses.mean_squared_error(y_true = y[i], y_pred = y_pred )
                loss   = tf.reduce_mean(loss)
            grads  = tape.gradient(loss, model.variables)
            optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
        print("epoch %d: loss %f" % (j+1, loss.numpy()))


epoch 1: loss 4171042.500000
epoch 2: loss 4129733.000000
epoch 3: loss 4075221.000000
epoch 4: loss 4010616.000000
epoch 5: loss 3936071.250000
epoch 6: loss 3848699.250000
epoch 7: loss 3747635.750000
epoch 8: loss 3632423.250000
epoch 9: loss 3503345.000000
epoch 10: loss 3360757.000000
epoch 11: loss 3205904.750000
epoch 12: loss 3040425.750000
epoch 13: loss 2866500.250000
epoch 14: loss 2686891.000000
epoch 15: loss 2504546.250000
epoch 16: loss 2322875.250000
epoch 17: loss 2145668.000000
epoch 18: loss 2145295.500000
epoch 19: loss 1938128.000000
epoch 20: loss 1777179.125000
epoch 21: loss 1634632.125000
epoch 22: loss 1503029.500000
epoch 23: loss 1361953.250000
epoch 24: loss 1253337.250000
epoch 25: loss 1158931.500000
epoch 26: loss 1030616.000000
epoch 27: loss 991843.812500
epoch 28: loss 969896.312500
epoch 29: loss 896037.125000
epoch 30: loss 826245.562500
epoch 31: loss 894608.187500
epoch 32: loss 775194.875000
epoch 33: loss 753607.687500
epoch 34: loss 732778.5000